In [ ]:
#@title Mount drive files
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
#@title Load libraries

import pandas as pd
import numpy as np


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/datacamp/GamesPrediction/train.zip')
test_data = pd.read_csv('/content/drive/MyDrive/datacamp/GamesPrediction/test.zip')

train_data['hist_games'] = train_data['historical_games'].apply(lambda x: list(map(int, x.split(" "))) )
test_data['hist_games'] = test_data['historical_games'].apply(lambda x: list(map(int, x.split(" "))) )


In [ ]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils.np_utils import to_categorical

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense


In [ ]:
def define_models(n_input, n_output, n_units):

    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,
                                                     initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

In [ ]:
train = train_data['hist_games']
label = train_data['next_game']

In [ ]:
def predict_sequence(infenc, infdec, source, n_steps, cardinality):

    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
      # predict next char
      yhat, h, c = infdec.predict([target_seq] + state)
      # store prediction
      output.append(yhat[0,0,:])
      # update state
      state = [h, c]
      # update target sequence
      target_seq = yhat
    return array(output)
 
 # decode a one hot encoded string
def one_hot_decode(encoded_seq):
	  return [argmax(vector) for vector in encoded_seq]

In [ ]:
n_input = 10
n_output = 5
train, infenc, infdec = define_models(n_input, n_output, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data preprocess
